# Ejercicio 4 elimina split y contrasplit en RV

- Ya tenemos los datos descargados, homogeneizados y limpios. Sin embargo es probable que tengamos que ajustar la serie. 
- Debemos buscar los split y contrasplit, calcular su efecto y ajustar la serie temporal, tantas veces como hayan sucedido.
- Una empresa puede valer 100€ teniendo 10 acciones de 10€ o teniendo 100 acciones de 1€. El precio de cotización varía drásticamente, pero el valor de la compañía no.
- Por desgracia, con los datos que nos hemos bajado, no disponemos ni del número de acciones, ni conocemos cuando han sucedido los split o contrasplit.
- Por otro lado, dentro de los datos que nos bajamos, sí que tenemos el cierre ajustado por dividendos; por lo que este problema nos lo dan resuelto.
- Deberemos trabajar con ese cierre ajustado.

- Objetivos:
    - Encuentra y anula los split y contrasplit
    - Programa una función a la que pasar un índice y que guarde en excel los datos del índice y de sus activos (homogeneizados, limpios y ajustados).
- Es decir 6 excels: Uno para el índice y 5 (High, Low, Volume, Adj_close y Open) para los activos, donde se agrupan los datos de todos los activos del índice.

Tiempo objetivo: 45 minutos

In [3]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
from datetime import timedelta
from time import mktime
import time
from tqdm import tqdm
import math
from rcurl import get_curl
from io import BytesIO
import pandas.io.formats.excel
import openpyxl
from openpyxl import load_workbook

In [4]:
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance  
    parameters: stock - short-handle identifier of the company    
    returns a tuple of header, crumb and cookie
    """   
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))
        output=(header, crumb[0], website.cookies)
        return output        

In [5]:
def convert_to_unix(date):
    """
    converts date to unix timestamp    
    parameters: date - in format (yyyy-mm-dd)    
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%Y-%m-%d')
    
    return int(mktime(datum.timetuple()))

convert_to_unix('2020-04-21')

In [6]:
def load_csv_data(stock, interval='1d', day_begin='20-03-2018', day_end='20-06-2018'):
    """
    queries yahoo finance api to receive historical data in csv file format
    
    parameters: 
        stock - short-handle identifier of the company        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)        
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    
    error1='404 Not Found: Timestamp data missing.'
    
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)   
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
        
        return website.text.split('\n')

In [7]:
def set_dates(x,ventana):
    
    """setup inicial de fechas,
    esta funcion devuelve la fecha de inicio
    y fin en string y formato YYYY-MM-DD
    Parametros:
    x=start o end
    ventana: entero referido al numero de dias del periodo"""
    
    hoy=datetime.now()
    if x=='end':
        fecha_fin = str(hoy.now())
        fecha_fin = fecha_fin[0:10]
        return (fecha_fin)
    if x=='start':
        fecha_inicial=hoy-timedelta(ventana)
        fecha_inicial = str(fecha_inicial)
        fecha_inicial = fecha_inicial[0:10]
        return (fecha_inicial)
    else:
        print('inputs incorrectos. Ver docstring d ela funcion')
        

In [8]:
def intentos_descarga_precios(stock_series,interval,fecha_inicial,fecha_fin, index_series, intentos):
    """funcion que en funcion del resultado de la descarga,
    devuelve un df con la informacion, y un codigo 0 'OK'
    o un codigo 1 de error y un DF vacio"""
    
    try:
        df = historic_prices_series(stock_series,interval,fecha_inicial,fecha_fin, index_series)
        codigo = 'OK'
        e = 'OK'
        return df, codigo, e
    
    except Exception as e:
        
        print('El activo no tiene activos descargables')
        print('Mensaje de error: ',str(e))
        print('Intento numero ',str(intentos))
        time.sleep(3)
        df = pd.DataFrame ({'dato':[],
                            'Divisa':[]})
        codigo = 'ERROR'
        return df, codigo, e

In [9]:
def historic_prices_series(stock_series,interval,fecha_inicial,fecha_fin, index_series):
    n=0
    x=0 
    for i in range(len(stock_series)):
        print("descargando "+stock_series[i]+' del indice '+index_series[i])
        try:
            DF=load_csv_data(stock_series[i], interval=interval, day_begin=fecha_inicial, day_end=fecha_fin)
            DF=pd.DataFrame(DF)
            DF = DF.iloc[:,0].str.split(",", n = 7, expand = True)
            DF.columns=DF.iloc[0,:]
            DF=DF.iloc[1:DF.shape[0],:]
            x=DF.shape[1]                 
            if x<2:
                 print("No nos hemos podido descargar el activo", benchmark)
            else:
                DF['stock'] = str(stock_series[i])
                DF['indice'] = str(index_series[i]) 
                DF=DF.dropna()
            
                if n==0:
                    dfacum=DF
                else:
                    dfacum=agrega_dataframes(DF,dfacum)    
                n=n+1
        except Exception as e:
            print("No nos hemos podido descargar el activo")
            print('key error:', e)
            next
        
    return dfacum


In [10]:
def agrega_dataframes(df,dfacum):
    """codigo que permite ir concatenando
    dataframes con estructura similar
    df:daframe a agregar
    dfacum: dataframe en donde se agregara"""
    
    dfacum=pd.concat([dfacum, df], axis=0,sort=True)
    dfacum.reset_index(drop=True)
    return dfacum

In [11]:
def split_df_acum_by_stock(dfacum):
    """codigo que permite ir desconcatenando
    dataframes con estructura similar a partir de un df acumulado
    requiere que la clave de desagregacion sea la columna 'stock'
    dfacum: dataframe en se ha agregado"""
    
    activos_a_importar = dfacum.stock.unique()
    for activo in activos_a_importar:
        globals()[activo] = dfacum[dfacum['stock']==activo]

In [12]:
def vector_lab_dates(fecha_inicial,ventana):
    """con esta funcion se obtiene un listado de fechas
    entre lunes y viernes, a partir de una fecha inicial
    y una ventana"""
    
    dates = pd.date_range(fecha_inicial, periods=ventana, freq='B')    
    inicio = datetime.strptime(fecha_inicial, '%Y-%m-%d')
    dias =ventana
    dates=[]
    
    for days in range(dias):
        date = inicio + timedelta(days=days)
        if date.weekday() < 5:
            dates.append(date)
            
    return pd.DataFrame({'dates':dates})

In [13]:
def dates_df_full(ventana, end, start):
    
    fecha_fin=set_dates('end',ventana)
    fecha_inicial=set_dates('start',ventana)
    print('la fecha de inicio es',fecha_inicial,' y la de fin es',fecha_fin)
    dates_df = vector_lab_dates(fecha_inicial, ventana)
    
    return dates_df

In [14]:
def componentes_indice_ind_download_0(ticker):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""
    
    lista_dato = []
    
    #primero se obtiene la divisa
    url="https://es.finance.yahoo.com/quote/"+ticker+"/components/"
    soup  = requests.get(url)
    soup  = BeautifulSoup(soup.content, 'html.parser')
    
    name_box = soup.find('span', attrs={'data-reactid': '4'})
    name = name_box.text.strip()
    divisa=name[len(name)-3:len(name)]
    
    print(ticker,divisa)
    soup  = requests.get(url)
    soup  = BeautifulSoup(soup.content, 'html.parser')
    
    name_box = soup.find_all('td')
    name_boxccc=name_box[0]
    name_empresa=name_box[1]
    name_boxccc = name_boxccc.get_text(strip=True)
    name_empresa = name_empresa.get_text(strip=True)
    
    for i in range(0,len(name_box)):
        name_boxccc=name_box[i]
        name_boxccc = name_boxccc.get_text(strip=True)
        lista_dato.append(name_boxccc)
    df=pd.DataFrame ({'dato':lista_dato})
    df['Divisa'] = divisa
    
    return df

In [15]:
def intentos_descarga_indice(ticker, intentos):
    """funcion que en funcion del resultado de la descarga,
    devuelve un df con la informacion, y un codigo 0 'OK'
    o un codigo 1 de error y un DF vacio"""
    
    try:
        df = componentes_indice_ind_download_0(ticker)
        codigo = 'OK'
        e = 'OK'
        print('El índice ', ticker,' se ha descargado correctamente')
        return df, codigo, e
    
    except Exception as e:
        print('El índice ', ticker,' no tiene activos descargables')
        print('Mensaje de error: ',str(e))
        print('Intento numero ',str(intentos))
        time.sleep(3)
        df = pd.DataFrame ({'dato':[],
                            'Divisa':[]})
        codigo = 'ERROR'
        return df, codigo, e

In [16]:
def componentes_indice_ind_download_1(ticker):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""
    
    intentos = 1
    for i in range(3):
        
        intentos_descarga_indice_res = intentos_descarga_indice(ticker, intentos)
        df = intentos_descarga_indice_res[0]
        codigo = intentos_descarga_indice_res[1]
        e = intentos_descarga_indice_res[2]
        
        if codigo == 'OK':
            return df
        
        if (codigo == 'ERROR') & (intentos <=3):
            intentos = intentos + 1
            time.sleep(3)
            
        if intentos ==3:
            print('Mensaje de error: ',str(e))
            print("Tras varios intentos no nos hemos podido descargar",ticker,"Dejamos de intentarlo.")
            df = pd.DataFrame ({'dato':[],
                                'Divisa':[]})
            return df

In [17]:
def recupera_valor_secuencia(df, start, freq, encabezado):
    valor = pd.DataFrame(np.arange(start, len(df), freq))
    valor['key'] = 1
    valor.index = valor[0]
    valor = valor.drop(0, 1)
    valor = pd.merge(df, valor, left_index=True, right_index=True)
    valor.reset_index(drop=True, inplace=True)
    valor.rename(columns={"dato": encabezado},inplace=True)
    valor = valor.drop(['key'], 1)
    return valor

In [18]:
def componentes_indice_ind_download_2(tickers_de_indices):
    """En este tercer paso, con la informacion descargada,reconstruimos la tabla. 
    Tiene 6 columnas (simbolo, nombre de la empresa, último precio, cambio, cambio % y volumen.)
    # Nos interesa obtener el símbolo y el nombre de la empresa.
    # Ojo, queremos importar únicamente los activos que tengan cotización. 
    """
    intentos = 0
    
    dfacum = pd.DataFrame ({'Activo':[],
                            'Divisa':[],
                            'Empresa':[],
                            'Precio':[],
                            'Indice':[]})
    
    for indice in tickers_de_indices:
        df = componentes_indice_ind_download_1(indice)
        Activo = recupera_valor_secuencia(df, 0, 6, 'Activo')
        df = df.drop(['Divisa'],1)
        Empresa = recupera_valor_secuencia(df, 1, 6, 'Empresa')
        Precio = recupera_valor_secuencia(df, 2, 6, 'Precio')
        df = pd.merge(Activo, Empresa, left_index=True, right_index=True)
        df = pd.merge(df, Precio, left_index=True, right_index=True)
        df['Indice'] = indice
        df = df[df.loc[:,'Precio'] != '']
        dfacum = agrega_dataframes(dfacum, df)
        dfacum.reset_index(drop=True, inplace=True)
        
    return dfacum

In [19]:
def depura_df_precios(dfacum):
    dfacum=dfacum.loc[:,['Adj Close','Close', 'Date','High','Low','Open','Volume','indice','stock']]
    (dfacum[(dfacum.Date).isna()]) = (dfacum[(dfacum.Date).isna()]).dropna()
    return dfacum

In [20]:
def dataframe_date_adj(dfacum, dates_df):
    """ajustamos por  fechas.
    las pasamos como indice y armonizamos fechas.
    dates_df han pasado por una funcion en donde se han depurado sabados y domingos
    dfacum: dataframe con los campos, Adj Close, Close, Date, High, Low, Open, Volume, indice,stock 
    dates_df: : dataframe con las fechas que seran la muestra final"""
    
    dfacum.index=dfacum.Date
    dates_df.index=dates_df.dates
    dates_df.index = pd.to_datetime(dates_df.index).strftime('%d-%m-%Y')
    dfacum.index = pd.to_datetime(dfacum.index).strftime('%d-%m-%Y')                            
    df_date_adj=pd.merge(dates_df, dfacum,left_index=True, right_index=True)
    df_date_adj = df_date_adj.drop(['dates'],1)
    df_date_adj = df_date_adj.drop(['Date'],1)
    
    df_date_adj.sort_index(axis = 0)
    return df_date_adj

In [21]:
def remplaza_nulos_na(df, registros_a_depurar):
    """Con esta funcion se rellenan datos no informados.
    En un primer paso, los nulos por nas, y en un segundo paso,
    los na por el dato de la fecha previa, y para los primeros daros de la 
    df: dataframe con los campos, Adj Close, Close, Date, High, Low, Open, Volume, indice,stock 
    activos: dataframe con los nombres de los activos a los que se aplica"""
    
    #Se identifican los los nulos y se reemplazan con NA
    df[df == 'null'] = np.nan
    df.replace('', np.nan)
    df.replace(' ', np.nan)
    df.replace('-', np.nan)
    df.replace('NA', np.nan)
    
    for activo in registros_a_depurar:
        df[activo] = df[activo].fillna(method='pad')
        df[activo] = df[activo].fillna(method='bfill')
        
    return df

In [22]:
def limpieza_de_datos(dfacum, dates_df):
    """funcion que agrega las rutinas definidas en el apartado para la depuracion de datos, 
    en base a fechas, y gestion de ausencias.
    como inputs el dataframe global y el dataframe de fechas laborables"""
    
    activos_a_importar = dfacum.stock.unique()
    
    for activo in activos_a_importar:
        globals()[activo] = dataframe_date_adj(globals()[activo], dates_df)
        globals()[activo] = remplaza_nulos_na(globals()[activo], globals()[activo].columns)

In [23]:
def genera_OCHLV_DF(dfacum, dates_df):
    
    activos_a_importar = dfacum.stock.unique()
    fechas=len(dates_df.index)
    nactivos=len(activos_a_importar)
    matriz=np.zeros((fechas, nactivos))
    carcasa=pd.DataFrame(matriz, index=dates_df.index, columns=activos_a_importar)
    Open = carcasa.copy()
    High = carcasa.copy()
    Low = carcasa.copy()
    Volume = carcasa.copy()
    Adj_Close = carcasa.copy()
    
    for activo in activos_a_importar:
        datos = globals().get(activo)
        Open[activo] = datos.loc[(datos['stock']==activo) == True , 'Open']
        High[activo] = datos.loc[(datos['stock']==activo) == True , 'High']
        Low[activo] = datos.loc[(datos['stock']==activo) == True , 'Low']
        Volume[activo] = datos.loc[(datos['stock']==activo) == True , 'Volume']
        Adj_Close[activo] = datos.loc[(datos['stock']==activo) == True , 'Adj Close']
        
    Open = remplaza_nulos_na(Open, Open.columns)
    High = remplaza_nulos_na(High, High.columns)
    Low = remplaza_nulos_na(Low, Low.columns)
    Volume = remplaza_nulos_na(Volume, Volume.columns)
    Adj_Close = remplaza_nulos_na(Adj_Close, Adj_Close.columns)
    
    return Open, High, Low, Adj_Close, Volume

In [24]:
def rentabilidades_diarias(df):
    
    # Los datos de cierre ajustados no están bien desmanipulados.
    # calculamos la rentabilidad de los activos
    
    rentabilidades = pd.DataFrame(0,
                                  index=df.index,
                                  columns=df.columns)
    
    # Ojo, los activos que no nos hemos podido descargar tienen precio de 0 €. La rentabilidad 0/0 dará un error que tenemos que evitar.
    rentabilidades = np.log(df.astype(float)).diff()
    rentabilidades = rentabilidades.dropna() #eliminamos la tipca primera fila de NaN
    
    return rentabilidades
    

In [25]:
def ajusta_serie_split(rentabilidades, Adj_Close, High, Low, Open):
    for activo in rentabilidades.columns:

    # Comprobamos si alguna rentabilidad de la serie es mayor o menor a 0.69 y calculamos los multiplicadores.
        indice_booleano = (rentabilidades[[activo]] > 0.69) | (rentabilidades[[activo]] < -0.69)
        rentabilidades_excesivas = rentabilidades[[activo]][indice_booleano.values]

        # Si la serie temporal no tiene ningún split o contra split no hay nada que hacer.
        if len(rentabilidades_excesivas > 0):
            multiplicador = pd.DataFrame(np.ones((len(Adj_Close), 1), dtype=int))
            print('Para el activo ',activo,' se han identificado Splits no ajustados bajo el enfoque de saltos del +-69% rentabilidad logaritmica')
            print('Se muestran registros afectados: ',rentabilidades_excesivas)

            # Recorremos las rentabilidades diarias para construir los multiplicadores.
            for dia in range(1, len(Adj_Close.columns) + 1):
                if rentabilidades.iloc[dia-1, :][[activo]][0] > 0.69:  # Han hecho un contra split. Ajustamos el multiplicador.
                    multiplicador.iloc[dia, 0] = multiplicador.iloc[dia-1, 0] * (Adj_Close.iloc[dia-1, :][[activo]] / Adj_Close.iloc[dia, :][[activo]])[0]
                elif rentabilidades.iloc[dia-1, :][[activo]][0] < -0.69:
                    multiplicador.iloc[dia, 0] = multiplicador.iloc[dia-1, 0] * (Adj_Close.iloc[dia-1, :][[activo]] / Adj_Close.iloc[dia, :][[activo]])[0]
                else:
                    multiplicador.iloc[dia, 0] = multiplicador.iloc[dia-1, 0]

            # Ajustamos los datos utilizando los multiplicadores.
            print('Se ha procedido al ajuste de la serie')

            Adj_Close[[activo]] = Adj_Close[[activo]] * multiplicador.values
            High[[activo]] = High[[activo]] * multiplicador.values
            Low[[activo]] = Low[[activo]] * multiplicador.values
            Open[[activo]] = Open[[activo]] * multiplicador.values

        else:
            print('Para el activo',activo,'no se han identificado Splits no ajustados bajo el enfoque de saltos del +-69% rentabilidad logaritmica')
            
    return Adj_Close, High, Low, Open

In [26]:
def save_OCHLV_fichero(Open, Adj_Close, High, Low, Volume, tipo, ExcelFile = 'Series_Ajustadas'):
    """Esta funcion salva los df asociados a .csv
    los DF: Open, Adj_Close, High, Low, Volume
    tipo:
    csv: genera 5 ficheros csv
    Excel: genera un libro excel con 5 pestanas"""
    
    if tipo == 'Excel':
        with pd.ExcelWriter(ExcelFile+'.xlsx') as writer:  
            Open.to_excel(writer, sheet_name="Open",index=True)
            Adj_Close.to_excel(writer, sheet_name="Adj_Close",index=True)
            High.to_excel(writer, sheet_name="High",index=True)
            Low.to_excel(writer, sheet_name="Low",index=True)
            Volume.to_excel(writer, sheet_name="Volume",index=True)
    else:
        Open.to_csv('Open.csv')
        Adj_Close.to_csv('Adj_Close.csv')
        High.to_csv('High.csv')
        Low.to_csv('Low.csv')
        Volume.to_csv('Volume.csv')

In [27]:
def algoritmo_lite(tickers_de_indices, ventana=60):
    
    dfacum = componentes_indice_ind_download_2(tickers_de_indices)
    fecha_fin=set_dates('end',ventana)
    fecha_inicial=set_dates('start',ventana)
    print('la fecha de inicio es',fecha_inicial,' y la de fin es',fecha_fin)
    dates_df = vector_lab_dates(fecha_inicial, ventana)
    dates_df_temp = dates_df

    dfacum = historic_prices_series(dfacum['Activo'],
                                            '1d', 
                                            fecha_inicial, 
                                            fecha_fin, 
                                            dfacum['Indice'])
    
    dfacum = depura_df_precios(dfacum)
    split_df_acum_by_stock(dfacum)
    activos_a_importar = dfacum.stock.unique()
    
    for activo in activos_a_importar:
        globals()[activo] = dfacum[dfacum['stock']==activo]
    print('se comienza la depuracion de datos')    
    
    limpieza_de_datos(dfacum, dates_df_temp)    
    print('depuracion de datos finalizada')
    
    OCHLV_DF = genera_OCHLV_DF(dfacum, dates_df)
    Open = OCHLV_DF[0]
    High = OCHLV_DF[1]
    Low = OCHLV_DF[2]
    Adj_Close = OCHLV_DF[3]
    Volume = OCHLV_DF[4]
    print('construccion de DataFrames de Adj_Close, High, Low, Open finalizada')

    rentabilidades = rentabilidades_diarias(Adj_Close)
    
    ajusta_serie_split_results = ajusta_serie_split(rentabilidades, Adj_Close, High, Low, Open)

    Adj_Close = ajusta_serie_split_results[0]
    High = ajusta_serie_split_results[1]
    Low = ajusta_serie_split_results[2]
    Open = ajusta_serie_split_results[3]
    
    print('Ajustes de series por split y contra split de de DataFrames de Adj_Close, High, Low, Open finalizada')
    save_OCHLV_fichero(Open, Adj_Close, High, Low, Volume, 'Excel') # Guardamos los DF finales en Excel.
    save_OCHLV_fichero(Open, Adj_Close, High, Low, Volume, 'csv') # Guardamos los DF finales en csv.
    
    print('Ficheros generados correctamente')   
    

In [28]:
tickers_de_indices = ("%5EBFX","%5EIBEX","%5EDJI")                    
                 
ventana = 500

In [29]:
algoritmo_lite(tickers_de_indices, ventana)

%5EBFX EUR
El índice  %5EBFX  se ha descargado correctamente
%5EIBEX EUR
El índice  %5EIBEX  se ha descargado correctamente
%5EDJI USD
El índice  %5EDJI  se ha descargado correctamente
la fecha de inicio es 2019-02-27  y la de fin es 2020-07-11
descargando UNH del indice %5EDJI
descargando MRK del indice %5EDJI
descargando JNJ del indice %5EDJI
descargando CSCO del indice %5EDJI
descargando V del indice %5EDJI
descargando AAPL del indice %5EDJI
descargando MCD del indice %5EDJI
descargando MSFT del indice %5EDJI
descargando HD del indice %5EDJI
descargando MMM del indice %5EDJI
descargando VZ del indice %5EDJI
descargando NKE del indice %5EDJI
descargando PFE del indice %5EDJI
descargando PG del indice %5EDJI
descargando CAT del indice %5EDJI
descargando INTC del indice %5EDJI
descargando DIS del indice %5EDJI
descargando IBM del indice %5EDJI
descargando WMT del indice %5EDJI
descargando RTX del indice %5EDJI
descargando KO del indice %5EDJI
descargando WBA del indice %5EDJI
descargan

### Para poder debugear,  dejo las funciones, paso a paso y explicadas

In [28]:
dfacum = componentes_indice_ind_download_2(tickers_de_indices)

%5EBFX EUR
El índice  %5EBFX  se ha descargado correctamente
%5EIBEX EUR
El índice  %5EIBEX  se ha descargado correctamente
%5EDJI USD
El índice  %5EDJI  se ha descargado correctamente


In [29]:
fecha_fin=set_dates('end',ventana)
fecha_inicial=set_dates('start',ventana)

print('la fecha de inicio es',fecha_inicial,' y la de fin es',fecha_fin)

la fecha de inicio es 2019-02-21  y la de fin es 2020-07-05


incluimos el tratamento de fechas del enunciado

In [30]:
dates_df = vector_lab_dates(fecha_inicial, ventana)

In [ ]:
dfacum = historic_prices_series(dfacum['Activo'],
                                '1d', 
                                fecha_inicial, 
                                fecha_fin, 
                                dfacum['Indice'])

descargando AAPL del indice %5EDJI
descargando TRV del indice %5EDJI
descargando GS del indice %5EDJI
descargando KO del indice %5EDJI
descargando HD del indice %5EDJI
descargando CSCO del indice %5EDJI
descargando UNH del indice %5EDJI
descargando VZ del indice %5EDJI
descargando BA del indice %5EDJI
descargando RTX del indice %5EDJI


In [ ]:
#algunas veces se descargan datos pero la info no es buena, generandose registros y columnas de suciedad

def depura_df_precios(dfacum):
    dfacum=dfacum.loc[:,['Adj Close','Close', 'Date','High','Low','Open','Volume','indice','stock']]
    (dfacum[(dfacum.Date).isna()]) = (dfacum[(dfacum.Date).isna()]).dropna()
    return dfacum    

In [ ]:
dfacum = depura_df_precios(dfacum)

In [ ]:
#Depuramos del DF de Suciedad

dfacum=dfacum.loc[:,['Adj Close','Close', 'Date','High','Low','Open','Volume','indice','stock']]
(dfacum[(dfacum.Date).isna()]) = (dfacum[(dfacum.Date).isna()]).dropna()

In [ ]:
split_df_acum_by_stock(dfacum)

In [ ]:
activos_a_importar = dfacum.stock.unique()

for activo in activos_a_importar:
    globals()[activo] = dfacum[dfacum['stock']==activo]

Volcamos los datos en un DF homogeneizado, utilizando la columna de fechas como índice.

Completamos los datos con NA (días en los que no hay registrada una cotización, pero que otras empresas sí cotizaron) na.locf localiza los NA del DF y los sustituye por el valor de la fila anterior. Si la fila con NA es la 1ª deja el NA sin dar un error.

utilizo esta solucion
  
  http://pyciencia.blogspot.com/2015/04/trabajar-con-datos-nan-en-dataframe.html
  
_Con los valores vecinos
Reemplazar los NaN con el valor anterior o posterior del NaN: 'pad' para reemplazarlo con el valor anterior y 'bfill' con el posterior. Se reemplazan todos los NaN, pero se puede establecer un límite según la distancia de este con el último dato del dataframe. Esta distancia se especifica en limit:_

In [ ]:
activos_a_importar = dfacum.stock.unique()

for activo in activos_a_importar:
    globals()[activo] = dataframe_date_adj(globals()[activo], dates_df)
    globals()[activo] = remplaza_nulos_na(globals()[activo], globals()[activo].columns)

In [ ]:
OCHLV_DF = genera_OCHLV_DF(dfacum, dates_df)
Open = OCHLV_DF[0]
High = OCHLV_DF[1]
Low = OCHLV_DF[2]
Adj_Close = OCHLV_DF[3]
Volume = OCHLV_DF[4]

In [ ]:
activos_a_importar = dfacum.stock.unique()
fechas=len(dates_df.index)
nactivos=len(activos_a_importar)
matriz=np.zeros((fechas, nactivos))
carcasa=pd.DataFrame(matriz, index=dates_df.index, columns=activos_a_importar)
Open = carcasa.copy()
High = carcasa.copy()
Low = carcasa.copy()
Volume = carcasa.copy()
Adj_Close = carcasa.copy()


Se crean los DF de metricas, en donde los activos son encabezados y las fechas filas.

In [ ]:
for activo in activos_a_importar:
    
    datos = globals().get(activo)
    Open[activo] = datos.loc[(datos['stock']==activo) == True , 'Open']
    High[activo] = datos.loc[(datos['stock']==activo) == True , 'High']
    Low[activo] = datos.loc[(datos['stock']==activo) == True , 'Low']
    Volume[activo] = datos.loc[(datos['stock']==activo) == True , 'Volume']
    Adj_Close[activo] = datos.loc[(datos['stock']==activo) == True , 'Adj Close']

Al asignar los activos a las fechas en base al indice, hemos ido muy rapido, 
pero se han podido generar algunos huecos adicionales al integrarlos en el DF.
En el ejercicio de R esto se hacia del tiron en el ejercicio anterior, pero en Python,
lo solvento trabajando con dataframes de menor tamaño hasta este punto, cruzar por indice,
y pasar de nuevo por la funcion de nas y nulos. 

No podemos hacer un dropna, ya que puede haber algun mercado abierto, por lo que pasamos la funcion de relleno de nulos

In [ ]:
Open = remplaza_nulos_na(Open, Open.columns)
High = remplaza_nulos_na(High, High.columns)
Low = remplaza_nulos_na(Low, Low.columns)
Volume = remplaza_nulos_na(Volume, Volume.columns)
Adj_Close = remplaza_nulos_na(Adj_Close, Adj_Close.columns)

In [ ]:
# Los datos de cierre ajustados no están bien desmanipulados.
# calculamos la rentabilidad de los activos

rentabilidades = pd.DataFrame(0,
                              index=Adj_Close.index,
                              columns=Adj_Close.columns)

# Ojo, los activos que no nos hemos podido descargar tienen precio de 0 €. La rentabilidad 0/0 dará un error que tenemos que evitar.
rentabilidades = np.log(Adj_Close.astype(float)).diff()
rentabilidades = rentabilidades.dropna() #eliminamos la tipca primera fila de NaN

por fin, llegamos a la parte final del ejercicio:

 Buscamos split y contrasplit, de existir, hay que "desmanipular" la serie de precios para hacerlos homogéneos entre sí.
    # Si la rent log es superior al 0.69 el precio se ha duplicado en un día, probablemente han realizado un contrasplit de la acción.
    # Si la rent log es inferior al -0.69 el precio se ha dividido entre do en un día, probablemente han realizado un split de la acción.

    #             Precio  Rentabilidad  Multiplicador       Precio desmanipulado
    # 25/8/16    100        -2.302        1                   100
    # 24/8/16   1000         2.302        100/1000            100
    # 23/8/16    100                      100/1000*1000/100   100

In [ ]:
Adj_Close = Adj_Close.astype(float)
High = High.astype(float)
Low = Low.astype(float)
Open = Open.astype(float)

ajusta_serie_split_results = ajusta_serie_split(rentabilidades, Adj_Close, High, Low, Open)

Adj_Close = ajusta_serie_split_results[0]
High = ajusta_serie_split_results[1]
Low = ajusta_serie_split_results[2]
Open = ajusta_serie_split_results[3]

In [ ]:
save_OCHLV_fichero(Open, Adj_Close, High, Low, Volume, 'Excel', 'Series_Ajustadas2') # Guardamos los DF finales en Excel.
save_OCHLV_fichero(Open, Adj_Close, High, Low, Volume, 'csv') # Guardamos los DF finales en csv.

## Otra forma de hacerlo

In [ ]:
import time
import re
import io
import numpy as np
import pandas as pd
import requests
from requests_html import HTMLSession
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
tickers_de_indices = [
    "%5EBFX", "%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE",
    "%5EGDAXI", "%5EHSI", "%5EIBEX", "%5EMXX", "%5EJKSE",
    "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI"
]

In [ ]:
ventana = 500

In [ ]:
def homogeneizar(datos, ventana):

    # Fechas inicio y fin
    fecha_inicial = (datetime.now() - relativedelta(days=ventana)).date()
    fecha_fin = datetime.now().date()

    # Creamos un vector con todas las fechas entre el día inicial y el final, sin fines de semana
    fechas = pd.date_range(start=fecha_inicial, end=fecha_fin, freq='B')

    # Creamos dataframe con la columna de fechas
    fechas = fechas.to_frame(index=False)
    fechas.columns = ['Date']

    # Volcamos los datos en un DF homogeneizado
    datos_homogeneizados = pd.merge(datos, fechas, how='outer', on='Date')

    # Manetemos orden descedente
    datos_homogeneizados.sort_values(by='Date', ascending=False, inplace=True)

    # Sustituir NA por el valor de la fila anterior. Si la fila con NA es la 1ª deja el NA sin dar un error.
    datos_homogeneizados.fillna(method='ffill', inplace=True)

    # Si los NA están en las primeras filas no hemos solucionado el problema. En principio no debería de haber más que dos (sábado y domingo), 
    # pero una empresa podría no cotizar lunes, martes... por lo que no conocemos el nº de potenciales NA a resolver.
    # Para resolver el problema, invertimos el orden del DF, aplicamos na.locf de nuevo y devolvemos el DF a su posición original.
    datos_homogeneizados.sort_values(by='Date', ascending=True, inplace=True)
    datos_homogeneizados.fillna(method='ffill', inplace=True)
    datos_homogeneizados.sort_values(by='Date', ascending=False, inplace=True)

    return datos_homogeneizados


In [ ]:
# Extraemos la función que intenta obtener los datos del índice.
def obtener_indice(url, cookies):

    try:
        benchmark = requests.get(url, cookies=cookies).text
        benchmark = io.StringIO(benchmark)
    except Exception as e:
        print("No nos hemos podido descargar el indice")
        print("Este es el mensaje de error que ha dado:")
        print(e)

        return("Sin datos")

    return benchmark

In [ ]:
# Función que intenta obtener los datos de cada activo
def obtener_datos_activo(activo, fecha_inicial, fecha_fin, columna_fechas):

    info_activo = pd.DataFrame()  # Inicializamos variable a dataframe vacío

    try:

        # html de la web del activo
        request = HTMLSession().get(f"https://es.finance.yahoo.com/quote/{activo}?ltr=1")
        res = request.text

        # Obtenemos el CrumbStore
        pattern_crumbstore = r'\"CrumbStore\"\:{\"crumb\":\"(?P<crumb>.*?)\"}'
        crumb = re.search(pattern_crumbstore, res).groupdict().get('crumb')
        crumb = crumb.replace("\n", "")  # Algunas veces, el crumb tiene una función de escape (un salto de página).

        # Construimos la url para bajarnos los datos
        url = f"https://query1.finance.yahoo.com/v7/finance/download/{activo}?period1={fecha_inicial}&period2={fecha_fin}&interval=1d&events=history&crumb={crumb}"

        # info activo dataframe
        html_text = HTMLSession().get(url, cookies=request.cookies).text
        info_activo = pd.read_csv(io.StringIO(html_text))

    except Exception as e:
        print(e)

    finally:

        if len(info_activo) == 0:
            print(f"No nos hemos podido descargar el activo {activo}")

            # Generamos un DF a ceros, con el tamaño adecuado, para que los siguientes pasos no den error.
            info_activo = pd.DataFrame(0,
                                       index=columna_fechas,
                                       columns=["Open", "High", "Low", "Close", "Adj Close", "Volume"])
            info_activo.index.name = 'Date'
            info_activo = info_activo.reset_index()  # Respetamos la misma estructura que el CSV que obtenemos de yahoo sin indice

        return info_activo

In [ ]:
def algoritmo_lite(indice, ventana=60):

    print(indice)

    # Extraemos el índice del listado de índices
    benchmark = indice

    # Establecemos el rango de fechas que queremos importar.
    fecha_inicial = (datetime.now() - relativedelta(days=ventana)).strftime('%s')
    fecha_fin = datetime.now().strftime('%s')

    # html de la web del benchmark
    request = HTMLSession().get(f"https://es.finance.yahoo.com/quote/{benchmark}?ltr=1")
    res = request.text

    # Obtenemos el CrumbStore
    pattern_crumbstore = r'\"CrumbStore\"\:{\"crumb\":\"(?P<crumb>.*?)\"}'
    crumb = re.search(pattern_crumbstore, res).groupdict().get('crumb')
    crumb = crumb.replace("\n", "")  # Algunas veces, el crumb tiene una función de escape (un salto de página).

    # Construimos la url para bajarnos los datos
    url = f"https://query1.finance.yahoo.com/v7/finance/download/{benchmark}?period1={fecha_inicial}&period2={fecha_fin}&interval=1d&events=history&crumb={crumb}"

    # Obtenemos benckmark
    intentos = 0
    benchmark = 'Sin datos'
    while benchmark == "Sin datos":

        benchmark = obtener_indice(url, request.cookies)

        intentos += 1
        if intentos > 3:
            benchmark == "No descargable"
            print("Tras varios intentos no nos hemos podido descargar los fondos. Dejamos de intentarlo.")
            return None

        if benchmark == "Sin datos":
            print("Error en la descarga de los fondos, reintentamos")
            time.sleep(30)

    benchmark = pd.read_csv(benchmark)

    # Hay veces que el último día se descarga dos veces en el DF. La segunda vez, la columna de volumen está a cero.
    if benchmark.iloc[-1, ]['Date'] == benchmark.iloc[-2, ]['Date']:
        benchmark = benchmark[:-1]

    # Cogemos la columna de fechas del índice para usarla cuando no nos consigamos bajar algún activo.
    columna_fechas = benchmark['Date']

    # Obtenemos los activos que contiene cada índice.
    intentos = 0
    descarga = "Sin datos"

    while descarga == "Sin datos":

        try:
            datos_web = HTMLSession().get(f"https://es.finance.yahoo.com/quote/{indice}/components/").html
            descarga = 'OK'
        except Exception:
            intentos = intentos + 1

            if intentos > 3:
                print("Tras varios intentos no nos hemos podido descargar los activos del índice. Dejamos de intentarlo.")
                return None

            if descarga == "Sin datos":
                print("Error en la descarga de los activos del índice, reintentamos")
                time.sleep(30)

    # activos del indice
    activos = pd.read_html(datos_web.find('table', first=True).html)[0]

    if len(activos) >= 1:  # Comprobamos si Yahoo publica algún activo del índice

        # Ojo, queremos importar únicamente los activos que tengan cotización.
        activos_con_datos = activos.dropna(subset=['Último precio'])

        # simbolo de cada empresa del índice
        activos_a_importar = activos_con_datos['Símbolo'].to_list()

        # nombre de cada empresa del índice
        nombres_empresas = activos_con_datos['Nombre de la empresa'].to_list()

        # divisa de las empresas del índice
        divisa_empresas = (datos_web
                           .find('#Col1-0-Components-Proxy', first=True)
                           .find('span', first=True)
                           .text
                           .split(' ')[-1])
    else:
        print(f"El índice {indice} no tiene activos descargables")


    # Importamos los activos que hemos seleccionado.
    for activo in activos_a_importar:

        intentos = 0
        descarga = "Sin datos"

        while descarga == "Sin datos":

            info_activo = obtener_datos_activo(activo, fecha_inicial, fecha_fin, columna_fechas)

            # Comprobamos que nos hemos podido descargar el activo.
            if len(info_activo) > 0:
                descarga = 'OK'

            intentos = intentos + 1

            if intentos > 3:
                print(f"Tras varios intentos no nos hemos podido descargar {activo} Dejamos de intentarlo.")
                descarga = 'OK'

            if descarga == "Sin datos":
                print(f"Error en la descarga del activo {activo} reintentamos")
                time.sleep(30)

        globals()[activo] = info_activo

    # Homogeneizamos los datos (que todos los DF tengan el mismo número de filas / días y que estén puestos en las mismas líneas)
    for activo in activos_a_importar:

        datos = globals().get(activo)

        # Comprobamos si nos hemos podido descargar el activo, o si nos hemos descargado más de un único día.
        if len(datos) >= 1:

            # Hay veces que el último día se descarga dos veces en el DF. La segunda vez, la columna de volumen está a cero.
            if datos.iloc[-1, ]['Date'] == datos.iloc[-2, ]['Date']:
                datos = datos[:-1]

            # Hay veces que hay datos duplicados entre medias del vector. Eliminamos las filas duplicadas.
            datos.drop_duplicates(subset="Date")

        # Confirmamos tipo date para la fecha
        datos['Date'] = pd.to_datetime(datos['Date'])
        datos_homogeneizados = homogeneizar(datos, ventana)

        # Guardamos el DF homogeneizado en su variable original.
        globals()[activo] = datos_homogeneizados

    # Creamos los DF donde guardaremos los resultados (excel puede contener algo más de 16.000 columnas)
    Open = pd.DataFrame(0, index=datos_homogeneizados['Date'], columns=activos_a_importar)
    High = pd.DataFrame(0, index=datos_homogeneizados['Date'], columns=activos_a_importar)
    Low = pd.DataFrame(0, index=datos_homogeneizados['Date'], columns=activos_a_importar)
    Volume = pd.DataFrame(0, index=datos_homogeneizados['Date'], columns=activos_a_importar)
    Adj_close = pd.DataFrame(0, index=datos_homogeneizados['Date'], columns=activos_a_importar)

    # Guardamos los datos en los DF finales
    for activo in activos_a_importar:

        datos = globals().get(activo)

        Open[activo] = datos['Open'].values
        High[activo] = datos['High'].values
        Low[activo] = datos['Low'].values
        Adj_close[activo] = datos['Adj Close'].values
        Volume[activo] = datos['Volume'].values

    # Los datos de cierre ajustados no están bien desmanipulados.
    # calculamos la rentabilidad de los activos
    rentabilidades = pd.DataFrame(0,
                                  index=Adj_close.index,
                                  columns=Adj_close.columns)

    # Ojo, los activos que no nos hemos podido descargar tienen precio de 0 €. La rentabilidad 0/0 dará un error que tenemos que evitar.
    rentabilidades = np.log(Adj_close).diff().shift(-1) * -1
    rentabilidades.iloc[-1, ] = rentabilidades.iloc[-2, ]

    # Aquellos que fueran todo 0s, tendremos NaN al hacer el np.log
    # Ponemos 0s en los NaN
    rentabilidades.fillna(0, inplace=True)

    # Buscamos split y contrasplit, de existir, hay que "desmanipular" la serie de precios para hacerlos homogéneos entre sí.
    # Si la rent log es superior al 0.69 el precio se ha duplicado en un día, probablemente han realizado un contrasplit de la acción.
    # Si la rent log es inferior al -0.69 el precio se ha dividido entre do en un día, probablemente han realizado un split de la acción.

    #             Precio  Rentabilidad  Multiplicador       Precio desmanipulado
    # 25/8/16    100        -2.302        1                   100
    # 24/8/16   1000         2.302        100/1000            100
    # 23/8/16    100                      100/1000*1000/100   100

    for activo in rentabilidades.columns:

        # Comprobamos si alguna rentabilidad de la serie es mayor o menor a 0.69 y calculamos los multiplicadores.
        indice_booleano = (rentabilidades[[activo]] > 0.69) | (rentabilidades[[activo]] < -0.69)
        rentabilidades_excesivas = rentabilidades[[activo]].loc[indice_booleano.values]

        # Si la serie temporal no tiene ningún split o contra split no hay nada que hacer.
        if len(rentabilidades_excesivas > 0):
            multiplicador = pd.DataFrame(np.ones((len(Adj_close), 1), dtype=int))

            # Recorremos las rentabilidades diarias para construir los multiplicadores.
            for dia in range(1, len(Adj_close.columns) + 1):
                if rentabilidades.iloc[dia-1, :][[activo]][0] > 0.69:  # Han hecho un contra split. Ajustamos el multiplicador.
                    multiplicador.iloc[dia, 0] = multiplicador.iloc[dia-1, 0] * (Adj_close.iloc[dia-1, :][[activo]] / Adj_close.iloc[dia, :][[activo]])[0]
                elif rentabilidades.iloc[dia-1, :][[activo]][0] < -0.69:
                    multiplicador.iloc[dia, 0] = multiplicador.iloc[dia-1, 0] * (Adj_close.iloc[dia-1, :][[activo]] / Adj_close.iloc[dia, :][[activo]])[0]
                else:
                    multiplicador.iloc[dia, 0] = multiplicador.iloc[dia-1, 0]

            # Ajustamos los datos utilizando los multiplicadores.
            Adj_close[[activo]] = Adj_close[[activo]] * multiplicador.values
            High[[activo]] = High[[activo]] * multiplicador.values
            Low[[activo]] = Low[[activo]] * multiplicador.values
            Open[[activo]] = Open[[activo]] * multiplicador.values

    # Guardamos los DF finales en csv.
    High.to_csv('High.csv')
    Low.to_csv('Low.csv')
    Volume.to_csv('Volume.csv')
    Adj_close.to_csv('Adj_close.csv')
    Open.to_csv('Open.csv')

    return None

In [ ]:
for indice in tickers_de_indices:
    
    # Invocamos a la función pasándola los índices uno a uno.
    recomendacion = algoritmo_lite(indice, ventana=60)